In [ ]:
from pathlib import Path
import datetime

import numpy as np
import polars as pl

import stock

In [ ]:
code_list = stock.kabutan.get_code_list()
current_date = datetime.date(year=2024, month=5, day=10)
start_date = current_date - datetime.timedelta(days=365)

In [ ]:
watch_list = []
for code in code_list:
    df = stock.kabutan.read_data_csv(code, start_date=start_date, end_date=current_date)
    fdf = stock.kabutan.read_financial_csv(code).filter(pl.col("annoounce_date") <= current_date).sort(pl.col("annoounce_date"))
    if len(fdf) > 0 and current_date - fdf["annoounce_date"][-1] < datetime.timedelta(days=1):
        continue

    # 過去10日の値動きの大きさを計算
    window_size = 10
    avg_key = "avg{}".format(window_size)
    stddev_key = "stddev{}".format(window_size)
    df = df.with_columns(
        pl.col("close").rolling_mean(window_size=window_size).alias(avg_key),
        pl.col("close").rolling_std(window_size=window_size).alias(stddev_key)
    )

    # ギャップアップしている
    df = df.with_columns(
        (pl.col("close") > pl.col(avg_key) + pl.col(stddev_key)).alias("breakpoint")
    )

    # 出来高が増加（急増）
    df = df.with_columns(
        pl.col("volume").rolling_max(window_size=window_size).shift().alias("max_volume")
    )
    df = df.with_columns(
        (pl.col("volume") > pl.col("max_volume") * 2).alias("volume_increase")
    )

    if len(df) > 0 and df["breakpoint"][-1] and df["volume_increase"][-1]:
        if (df["close"][-1] - df["low"][-1]) / max(df["high"][-1] - df["low"][-1], 1e-5) > 0.8:
            if stock.kabutan.data.get_market_capitalization(code) < 1000:
                watch_list.append(code)


In [ ]:
len(watch_list)

In [ ]:
watch_list

In [ ]:
pl.Config.set_tbl_rows(-1)
df.filter((pl.col("breakpoint") == True) & (pl.col("volume_increase") == True))